In [10]:
# -*- coding: utf-8 -*-

# 东方财富网国内新闻数据抓取器
## 拂晓工作室
### 此程序将爬取某个板块所有最近的新闻并存进CSV！

# 参考资料
* [1] BeautifulSoup的例子：https://qiita.com/rusarusa/items/d7f014ba80d6fe7a3e07
* [2] BeautifulSoup的文档：https://www.cnblogs.com/zhaof/p/6930955.html
* [3] 暴力安装BS4: https://github.com/ipython/ipython/issues/10684
* [4] CSV文档：https://docs.python.org/2/library/csv.html
* [5] CSV使用实例：https://blog.csdn.net/waple_0820/article/details/70049953
* [6] 如何解决list重复元素的问题：https://stackoverflow.com/questions/7961363/removing-duplicates-in-lists
* [7] 解决CSV文件中文乱码的问题：https://blog.csdn.net/fwj_ntu/article/details/78563962

In [11]:
# -*- coding: utf-8 -*-
# 导入库
import urllib
import re
import pandas as pd
import pymysql
import os
import turicreate as tc
import csv
from bs4 import BeautifulSoup


In [12]:
# 新闻板块(同时也会加入热门新闻的数据)
china = 'cgnjj'  # 国内
international = 'cgjjj'  # 国际

# 定义基本常量
url = 'http://finance.eastmoney.com/news/' + china + '.html' # 主目录地址 更换不同主目录以爬取不同板块内容 (BAD)
csv_filepath = '../Datasets/Eastmoney/news/'  # 存储数据文件地址

In [13]:
# 创建一个二十五页的list
page_list = []
counter = 1

while counter <= 25:
    pageurl = 'http://finance.eastmoney.com/news/' + china
    if counter != 1:
        pageurl = pageurl + '_' + str(counter) + '.html'
        page_list.append(pageurl)
    else:
        pageurl = pageurl + '.html'
        page_list.append(pageurl)
    counter += 1
    
print('成功创建包含 ' + str(len(page_list)) + ' 个页面链接的目录！')

成功创建包含 25 个页面链接的目录！


In [14]:
# 创建一个包含20个class的list
counter = 0
class_list = []

while counter < 20:
    class_list.append('newsTr' + str(counter))
    counter += 1
    
print('成功创建包含 ' + str(len(class_list)) + ' 个class的目录！')
# print(class_list)  # Debug

成功创建包含 20 个class的目录！


In [15]:
# 初始化urllist
urllist = []
year = '2018'  # Separate parameter
counter = 1

# 创造urllist
print('-----------------获取所有文章超链接程序-----------------')
print('分析任务开始！')
for url in page_list:
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    for each_url in soup.find_all('a', href=True):
        if 'http://finance.eastmoney.com/news/' in each_url['href']:
            if year in each_url['href']:
                urllist.append(each_url['href'])   
    print('第' + str(counter) + '页分析完成！现在一共有' + str(len(urllist)) + '篇文章的数据！')
    counter += 1

urllist = list(set(urllist))
print('全部分析完成！正在查重...')
print('任务完成！共获得了' + str(len(urllist)) + '篇文章的数据！')
print('---------------------------------------------')

-----------------获取所有文章超链接程序-----------------
分析任务开始！


KeyboardInterrupt: 

In [7]:
# 抓取新闻URL函数
def collectNews(url, counter):
    # 准备工作
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    
    # 获得文章的发表时间
    time = soup.find(class_="time").get_text()
    
    # 获得文章的内容
    content = soup.find(id="ContentBody").get_text() 
    content = content[0:re.search('责任编辑', content).span()[0]-1].strip()
    print content
    
    # 获得相关的股票单
    related_stocks = []
    for each in soup.find_all(class_='keytip'): 
        related_stocks.append(each.get_text())
    
    # 将list处理成String 中间用&分割
    related_stocks_string = ''
    for stock in related_stocks:
        related_stocks.append('&')
        related_stocks.append(stock)
        
    # 写入csv
    with open("../DataSets/Eastmoney/news/RecentStockNews.csv","w") as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow([counter, 
                         time[0:4], time[0:4], time[8:10], 
                         soup.find('h1').get_text(), 
                         content, 
                         related_stocks_string])
    # 内存回收
    del req
    del response
    del html
    del soup
    del time
    del content
    del related_stocks
    del related_stocks_string
    del writer
    
    # 刷新计数
    counter += 1
    return counter

In [8]:
# 创建csv文件
with open("../DataSets/Eastmoney/news/RecentStockNews.csv", "w") as csvfile: 
    writer = csv.writer(csvfile)

    # 先写入columns_name
    writer.writerow(["index", "year", "month", "day", "title", "contents", "related_stocks"])

In [ ]:
# Index Counter
counter = 1
total = len(urllist)

# 扒取资料
for each_article in urllist:
    print('正在分析第' + str(counter) + '篇文章，共' + str(total) + '篇.')
    counter = collectNews(each_article, counter)

正在分析第1篇文章，共516篇.


# =========================TRASHYARD=========================

In [29]:
# 抓取新闻URL函数
def collectNews(url, counter):
    # 准备工作
    req = urllib.request.Request(url)
    response = urllib.request.urlopen(req)
    html = response.read()
    soup = BeautifulSoup(html, "lxml")
    
    # 获得文章的标题
    title = soup.find('h1').get_text()
    # print(title)  # Debug
    
    # 获得文章的发表时间
    time = content = soup.find(class_="time").get_text()
    year = time[0:4]
    month = time[5:7]
    day = time[8:10]
    # print(time,year,month,day)  # Debug
    
    # 获得文章的内容
    content = soup.find(id="ContentBody").get_text()
    index = re.search('责任编辑', content).span()[0]-1
    content = content[0:index].strip()
    # print(content)  # Debug
    
    # 获得相关的股票单
    related_stock = soup.find_all(class_='keytip')
    related_stocks = []
    for each in related_stock:
        stock = each.get_text()
        related_stocks.append(stock)
        # print(each.get('href'))  #大概不需要URL
    # print(related_stocks)  # Debug
    
    # 将list处理成String 中间用&分割
    related_stocks_string = ''
    for stock in related_stocks:
        related_stocks.append('&')
        related_stocks.append(stock)
        
    # 写入csv
    with open("../DataSets/Eastmoney/news/RecentStockNews.csv","w") as csvfile: 
        writer = csv.writer(csvfile)
        writer.writerow([counter, year, month, day, title, content, related_stocks_string])
    
    # 刷新计数
    counter += 1
    return counter